# Sequence to Sequence model to extract summary,objective from resume (input: entire resume)


## 207 resumes


In [28]:
from __future__ import print_function
import csv,os
import collections
from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=32
epochs=25
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv"

## vectorizing the data

In [37]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
count=0
for file in files:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        input_text=" "
        output_text=" "
        count=0
        for Input_text,output in reader:
            Input_text=Input_text.strip().lower()
            input_text+=Input_text+' \n'
            if(output=='1'):
                output_text+=Input_text+' \n'
            
            if count==15:
                break
            count+=1
            
    """
        print("*"*50)
        print(" "*50)
        #print(input_text)
        print("-"*50)
        print(output_text)
        print(" "*50)
        print("*"*50)
    if count==2:
        break
    count+=1
    """

    # We use "<SOL>" as the "start sequence" character
    # for the targets, and "<EOL>" as "end sequence" character.
    output_text='<SOL> '+output_text+' <EOL>'
    input_resumes.append(input_text)
    output_summary.append(output_text)
    for word in input_text.split():
        if word not in input_tokens:
            input_tokens[word]+=1
    for word in output_text.split():
        if word not in output_tokens:
            output_tokens.add(word)


In [38]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),3000)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


5638
Number of samples: 208
number of unique input token: 3000
number of unique output token: 3243
Max Sequence length for inputs: 178
Max Sequence length for outputs: 155


## defining token2index

In [39]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

## defing encoder_input,decoder_input and decoder_output

In [40]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [41]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
        
    for t,word in enumerate(target_text.split()):
        decoder_input_data[i,t,output_token2index[word]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,output_token2index[word]]=1


## Define an input sequence and process it.

In [42]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [26]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 4000)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None, 3243)   0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 128), (None, 2114048     input_7[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 128),  1726464     input_8[0][0]                    
                                                                 lstm_5[0][1]                     
          

## function to test model

In [43]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = dict(
    (i, word) for word, i in input_token2index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in output_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token2index['<SOL>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<EOL>' or
           len(decoded_sentence.split()) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    if('<EOL>' in decoded_sentence):
        decoded_sentence=" ".join(decoded_sentence.split()[:-1])
    return decoded_sentence


## Run training

In [19]:

iteration=0
"""
# load weights
print('loading the weights')
model=load_model('resume_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(1):
    index=np.random.randint(len(input_resumes))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print("-"*50)
    print(input_resumes[index])
    print(" "*50)
    print("*"*50+"\nOUTPUT"+" "*50)
    print(output_sequence)
    print("-"*50+"\n"+" "*50)
"""
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/iteration_resume_level.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="resume_level_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(1):
        index=np.random.randint(len(input_resumes))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print("-"*50)
        print(input_resumes[index])
        print(" "*50)
        print("*"*50+"\nOUTPUT"+" "*50)
        print(output_sequence)
        
        
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('resume_level.h5')

file_data, iteration:12
Iteration: 13
Train on 166 samples, validate on 42 samples
Epoch 1/25


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-b348d1432f31>", line 46, in <module>
    validation_split=0.2,callbacks=callbacks_list)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training.py", line 1042, in fit
    validation_steps=validation_steps)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2653, in __call__
    if hasattr(get_session(), '_make_callable_from_options'):
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 203, in get_session
    session.run(tf.variables_initi

KeyboardInterrupt: 